## 寻找市场上的强势股

In [18]:
import a1chemy.data_source as data_source
import pymongo
import operator
from tqdm.notebook import tqdm

sorted_roc_list = sorted(roc_list, key = operator.itemgetter('C/S', 'S/M', 'M/L'))

mongo_client = pymongo.MongoClient("mongodb://localhost:27017/", username='a1chemy', password='1B2C9046-E3CC-447F-9961-E125759BA44F')
mongo_ticks_client = data_source.MongoTicks(mongo_client)
mongo_tags_client = data_source.MongoTags(mongo_client)

stocks_tree = mongo_tags_client.tree(id='ZH_ETF')
stocks = []
for id, stock_tag in tqdm(stocks_tree.root.children.items()):
    exchange=stock_tag.values['exchange']
    symbol =stock_tag.values['symbol']
    print("stock info: exchange={}, symbol={}".format(exchange, symbol))
    ticks = mongo_ticks_client.find_one(exchange=exchange, symbol=symbol)
    stocks.append(ticks)

stock info: exchange=SH, symbol=SH515650
stock info: exchange=SH, symbol=SH588000
stock info: exchange=SH, symbol=SH513050
stock info: exchange=SH, symbol=SH512070
stock info: exchange=SH, symbol=SH512290
stock info: exchange=SZ, symbol=SZ159938
stock info: exchange=SZ, symbol=SZ159992
stock info: exchange=SH, symbol=SH512170
stock info: exchange=SH, symbol=SH511010
stock info: exchange=SH, symbol=SH511260
stock info: exchange=SH, symbol=SH501050
stock info: exchange=SZ, symbol=SZ159928
stock info: exchange=SH, symbol=SH515030
stock info: exchange=SH, symbol=SH510330
stock info: exchange=SZ, symbol=SZ159997
stock info: exchange=SZ, symbol=SZ159995
stock info: exchange=SH, symbol=SH512760
stock info: exchange=SH, symbol=SH512480
stock info: exchange=SH, symbol=SH510230
stock info: exchange=SH, symbol=SH512400
stock info: exchange=SH, symbol=SH512800
stock info: exchange=SH, symbol=SH512880
stock info: exchange=SH, symbol=SH512000
stock info: exchange=SH, symbol=SH512660
stock info: exch

In [19]:
import pandas as pd

def roc(ticks):
    def rate(t, day1, day2):
        length = t.raw_data.shape[0]
        if -length >= day1 or -length >= day2:
            return -100
        return 100*(t.close().iloc[day1] - t.close().iloc[day2])/t.close().iloc[day2]
    
    return {
        'name': ticks.name,
        'symbol': ticks.symbol,
        '1D': rate(ticks, -1, -2),
        '5D': rate(ticks, -1, -6),
        '1M': rate(ticks, -1, -21),
        'C/S': rate(ticks, -1, -21),
        'S/M': rate(ticks, -21, -61),
        'M/L': rate(ticks, -61, -121),
    }

roc_list = []
for stock_tick in stocks:
    roc_list.append(roc(stock_tick))
df=pd.DataFrame(sorted_roc_list)

def percent_color(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if val < 0 else 'green'
    return 'color: %s' % color
# df.style
percent_list = list(df)[2:]
df.style.format("{:.2f}", subset=percent_list).applymap(percent_color, subset=percent_list)

,name,symbol,1D,5D,1M,C/S,S/M,M/L
0,科创50ETF,SH588000,1.29,-0.49,-100.00,-100.00,-100.00,-100.00
1,医疗ETF,SH512170,3.01,0.68,-8.10,-8.10,1.70,29.67
2,生物医药ETF,SH512290,2.29,0.59,-7.55,-7.55,-6.59,31.85
3,创新药,SZ159992,3.00,0.95,-6.54,-6.54,-6.59,31.86
4,医药,SZ159938,2.35,0.82,-5.17,-5.17,-5.53,28.90
5,中概互联网ETF,SH513050,-0.38,-1.13,-3.04,-3.04,2.07,28.86
6,科技50ETF,SH515750,1.54,-0.21,-1.76,-1.76,-3.52,25.12
7,十年国债ETF,SH511260,-0.07,0.06,-0.61,-0.61,0.51,-1.59
8,国债ETF,SH511010,-0.15,0.06,-0.44,-0.44,0.54,-1.80
9,消费ETF,SZ159928,0.80,-2.03,1.25,1.25,-6.19,41.09
